In [1]:
import os, pathlib

file_path = os.path.dirname(os.path.realpath("__file__"))
home_dir = pathlib.Path(file_path).parent.parent.parent
os.chdir(home_dir)

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from src.utils import get_optimizer, SAVED_CLASSIFIER_PATH_FASHION, keras_dataset_image_preprocessing, DENSE_LAYER_NAME

from src.vae import ClaDecVAE

tf.random.set_seed(1234)

(train_images, train_labels_c), (test_images, test_labels_c) = keras_dataset_image_preprocessing(
    keras.datasets.fashion_mnist)
# get the trained model and check accuracy on test data for later reference
classifier = keras.models.load_model(SAVED_CLASSIFIER_PATH_FASHION)

2022-07-09 14:32:44.627667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-09 14:32:44.675856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-09 14:32:44.676097: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-09 14:32:44.677013: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
classifier.evaluate(test_images, test_labels_c)
classifier.summary()

2022-07-09 14:32:47.236010: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


313/313 [==============================] - 3s 4ms/step - loss: 0.2533 - categorical_accuracy: 0.9101
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 64)       256       
 hNormalization)     

In [4]:
alpha = 0.9
latent_dim = 2
optimizer = get_optimizer()
claDec = ClaDecVAE(classifier, DENSE_LAYER_NAME, alpha, latent_dim)
claDec.compile(optimizer=get_optimizer())

TypeError: Can't instantiate abstract class ClaDecVAE with abstract method test_step

In [ ]:
claDec.fit(train_images, train_labels_c, epochs=10)

In [ ]:
classifier.evaluate(test_images, test_labels_c)  # classifier weights not altered

In [ ]:
import matplotlib.pyplot as plt


def plot_latent_space(vae, n=30, figsize=30):
    # display a n*n 2D manifold of digits
    digit_size = 28
    scale = 1.0
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = vae.decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[
            i * digit_size: (i + 1) * digit_size,
            j * digit_size: (j + 1) * digit_size,
            ] = digit

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent_space(claDec)

In [ ]:
def plot_label_clusters(vae, data, classes):
    # display a 2D plot of the digit classes in the latent space
    acts = vae.model_up_to_layer_to_explain(data)
    z_mean, _, _ = vae.encoder.predict(acts)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=classes)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()


test_labels = np.argmax(test_labels_c, axis=1)
train_labels = np.argmax(train_labels_c, axis=1)
encoding = ["T-Shirt (Top)", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]
plot_label_clusters(claDec, train_images[:10000], train_labels[:10000])

In [ ]:
plot_label_clusters(claDec, test_images, test_labels)